<a href="https://colab.research.google.com/github/sherryjasal/ASR/blob/main/Fine_Tuning_ASR_Custom_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Loading Libraries**

In [ ]:
!pip3 install datasets
!pip3 install torchaudio
!pip3 install librosa
!pip3 install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 39.3 MB/s eta 0:00:00


In [ ]:
!pip3 install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-v4jq_up0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v4jq_up0
  Resolved https://github.com/huggingface/transformers to commit 2cf87e2bbb46ff7ec9dd2746d694d41b6815fb43
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.33.0.dev0-py3-none-any.whl size=7563075 sha256=3cd13d62c50e84a53e4bd902d8d9673b9f251e71de3a3bfb1cadd063b6e7d241
  Stored in directory: /tmp/pip-ephem-wheel-cache-fchtyt3p/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [ ]:
!pip3 install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00


In [ ]:
#!pip3 install transformers --upgrade

In [ ]:
import transformers
print(transformers.__version__)

4.33.0.dev0


# **Loading Dataset and Creating Dataset**

In [ ]:
import os

# Define paths for the provided data
audio_files = sorted([f"/content/drive/MyDrive/mysample/mysample/WAV/customer_service_05000{i}.wav" for i in range(1, 6)])
text_files = sorted([f"/content/drive/MyDrive/mysample/mysample/TEXT/customer_service_05000{i}.txt" for i in range(1, 6)])

# Read the text files to get the transcripts
transcripts = []
for text_file in text_files:
    with open(text_file, 'r') as f:
        transcripts.append(f.read().strip())

# Create a dataset by pairing audio files with transcripts
dataset = [{"path": audio, "text": transcript} for audio, transcript in zip(audio_files, transcripts)]
dataset


[{'path': '/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050001.wav',
  'text': 'الهيئة العامة للسياحة والآثار'},
 {'path': '/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050002.wav',
  'text': 'خدمة إبداء رغبة في مشروع الفنادق التراثية ترحب بكم.'},
 {'path': '/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050003.wav',
  'text': 'من فضلك، اذكر اسم الشركة.'},
 {'path': '/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050004.wav',
  'text': 'من فضلك اذكر تاريخ تأسيس الشركة.'},
 {'path': '/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050005.wav',
  'text': 'من فضلك اذكر موقع الشركة الإلكتروني.'}]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.DataFrame(dataset)

In [ ]:
# Split the dataset into train and test sets (90% train, 10% test)
train_data, test_data = train_test_split(df, test_size=0.10, random_state=42)

train_data, test_data


(                                                path  \
 4  /content/drive/MyDrive/mysample/mysample/WAV/c...   
 2  /content/drive/MyDrive/mysample/mysample/WAV/c...   
 0  /content/drive/MyDrive/mysample/mysample/WAV/c...   
 3  /content/drive/MyDrive/mysample/mysample/WAV/c...   
 
                                    text  
 4  من فضلك اذكر موقع الشركة الإلكتروني.  
 2             من فضلك، اذكر اسم الشركة.  
 0         الهيئة العامة للسياحة والآثار  
 3      من فضلك اذكر تاريخ تأسيس الشركة.  ,
                                                 path  \
 1  /content/drive/MyDrive/mysample/mysample/WAV/c...   
 
                                                 text  
 1  خدمة إبداء رغبة في مشروع الفنادق التراثية ترحب...  )

# **Creating Vocabulary**

In [ ]:
all_sentences = [str(sentence) for sentence in train_data['text']] + [str(sentence) for sentence in test_data['text']]

In [ ]:
def extract_all_characters(data):
    entire_text = " ".join(data)
    vocabulary = list(set(entire_text))
    return sorted(vocabulary)

vocab = extract_all_characters(all_sentences)

vocab

[' ',
 '.',
 '،',
 'ء',
 'آ',
 'أ',
 'إ',
 'ئ',
 'ا',
 'ب',
 'ة',
 'ت',
 'ث',
 'ح',
 'خ',
 'د',
 'ذ',
 'ر',
 'س',
 'ش',
 'ض',
 'ع',
 'غ',
 'ف',
 'ق',
 'ك',
 'ل',
 'م',
 'ن',
 'ه',
 'و',
 'ي']

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab)}
vocab_dict

{' ': 0,
 '.': 1,
 '،': 2,
 'ء': 3,
 'آ': 4,
 'أ': 5,
 'إ': 6,
 'ئ': 7,
 'ا': 8,
 'ب': 9,
 'ة': 10,
 'ت': 11,
 'ث': 12,
 'ح': 13,
 'خ': 14,
 'د': 15,
 'ذ': 16,
 'ر': 17,
 'س': 18,
 'ش': 19,
 'ض': 20,
 'ع': 21,
 'غ': 22,
 'ف': 23,
 'ق': 24,
 'ك': 25,
 'ل': 26,
 'م': 27,
 'ن': 28,
 'ه': 29,
 'و': 30,
 'ي': 31}

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

34

In [ ]:
import json

with open('vocab.json', 'w', encoding='utf-8') as vocab_file:
    json.dump(vocab_dict, vocab_file)


# **Tokenizing Data**

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
processor.save_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-sample_data")

In [ ]:
tokenizer_directory = tokenizer.save_pretrained("/content/drive/MyDrive/tokenizer_asr")

In [ ]:
train_data['path'][0]

'/content/drive/MyDrive/mysample/mysample/WAV/customer_service_050001.wav'

# **Loading & Resampling Data**

In [ ]:
import torchaudio

def load_and_resample(audio_path, target_sampling_rate=16000):
    """Load an audio file and resample it to the target sampling rate."""
    waveform, sampling_rate = torchaudio.load(audio_path)
    if sampling_rate != target_sampling_rate:
        resampler = torchaudio.transforms.Resample(orig_freq=sampling_rate, new_freq=target_sampling_rate)
        waveform = resampler(waveform)
    return waveform


# **Preprocessing Data**

In [ ]:
# Process your training and testing audio data
train_audio_paths = train_data['path']
test_audio_paths = test_data['path']

train_audio = [load_and_resample(path) for path in train_audio_paths]
test_audio = [load_and_resample(path) for path in test_audio_paths]

In [ ]:
train_audio[0]

tensor([[ 0.0001,  0.0005,  0.0007,  ...,  0.0004,  0.0002, -0.0002]])

In [ ]:
print(type(train_audio[0]))
print(train_audio[0].shape)

<class 'torch.Tensor'>
torch.Size([1, 58454])


In [ ]:
train_audio = [tensor.squeeze(0) for tensor in train_audio]
test_audio = [tensor.squeeze(0) for tensor in test_audio]

In [ ]:
print(type(train_audio[0]))
print(train_audio[0].shape)

<class 'torch.Tensor'>
torch.Size([58454])


In [ ]:
train_audio = [tensor.squeeze(0).numpy() for tensor in train_audio]
test_audio = [tensor.squeeze(0).numpy() for tensor in test_audio]

In [ ]:
print(type(train_audio[0]))
print(train_audio[0].shape)

<class 'numpy.ndarray'>
(58454,)


In [ ]:
audio_max_length = 16000 * 10

train_audio_processed = processor.feature_extractor(train_audio, return_tensors="pt", padding="longest", truncation=True, max_length=audio_max_length, sampling_rate=16000)
test_audio_processed = processor.feature_extractor(test_audio, return_tensors="pt", padding="longest", truncation=True, max_length=audio_max_length, sampling_rate=16000)

In [ ]:
# Extract text values from the dataframes
train_texts = train_data['text'].tolist()
test_texts = test_data['text'].tolist()

In [ ]:
# Ensure all elements are strings and filter out any None values
train_texts = [str(text) for text in train_texts if text is not None]
test_texts = [str(text) for text in test_texts if text is not None]

#train_texts, test_texts

In [ ]:
# Tokenize texts
train_labels = tokenizer(train_texts, return_tensors="pt", padding="longest").input_ids
test_labels = tokenizer(test_texts, return_tensors="pt", padding="longest").input_ids

In [ ]:
train_dataset = {"input_values": train_audio_processed["input_values"], "labels": train_labels}
test_dataset = {"input_values": test_audio_processed["input_values"], "labels": test_labels}

#train_dataset, test_dataset

In [ ]:
def reformat_dataset(dataset):
    return [{"input_values": audio, "labels": label} for audio, label in zip(dataset["input_values"], dataset["labels"])]

train_dataset = reformat_dataset(train_dataset)
test_dataset = reformat_dataset(test_dataset)


In [ ]:
print(train_dataset[0])

{'input_values': tensor([ 0.0030,  0.0109,  0.0162,  ...,  0.0098,  0.0052, -0.0046]), 'labels': tensor([27, 28, 32, 23, 20, 26, 25, 32,  8, 16, 25, 17, 32, 27, 30, 24, 21, 32,
         8, 26, 19, 17, 25, 10, 32,  8, 26,  6, 26, 25, 11, 17, 30, 28, 31,  1])}


In [ ]:
print(test_dataset[0])

{'input_values': tensor([ 0.0044,  0.0055,  0.0046,  ..., -0.0072, -0.0040,  0.0056]), 'labels': tensor([14, 15, 27, 10, 32,  6,  9, 15,  8,  3, 32, 17, 22,  9, 10, 32, 23, 31,
        32, 27, 19, 17, 30, 21, 32,  8, 26, 23, 28,  8, 15, 24, 32,  8, 26, 11,
        17,  8, 12, 31, 10, 32, 11, 17, 13,  9, 32,  9, 25, 27,  1])}


In [ ]:
#print(len(train_dataset))

# **Setting Up Trainer**

In [ ]:
!pip3 install git+https://github.com/huggingface/accelerate

  Cloning https://github.com/huggingface/accelerate to /tmp/pip-req-build-l7r9soo5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /tmp/pip-req-build-l7r9soo5
  Resolved https://github.com/huggingface/accelerate to commit 812719644d15576b2279164dd5f3cb020595d047
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for accelerate: filename=accelerate-0.23.0.dev0-py3-none-any.whl size=251285 sha256=4cd99eba256cccc94d2a97d5bc2e0ff8537b2c74a37593e7199c48f46b00dacb
  Stored in directory: /tmp/pip-ephem-wheel-cache-fx8eqdjv/wheels/f6/c7/9d/1b8a5ca8353d9307733bc719107acb67acdc95063bba749f26
Successfully built accelerate
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.22.0
    Uninstalling accelerate-0.22.0:
      Successfully uninstalled accelerate-0.22.0


In [ ]:
!pip3 install accelerate>=0.20.1 -U

In [ ]:
from transformers import Wav2Vec2Processor
from typing import List, Dict, Union
import torch

class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    """

    def __init__(self, processor: Wav2Vec2Processor, padding=True, max_length=None, max_length_labels=None, pad_to_multiple_of=None, pad_to_multiple_of_labels=None):
        self.processor = processor
        self.padding = padding
        self.max_length = max_length
        self.max_length_labels = max_length_labels
        self.pad_to_multiple_of = pad_to_multiple_of
        self.pad_to_multiple_of_labels = pad_to_multiple_of_labels

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors="pt")

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(label_features, padding=self.padding, max_length=self.max_length_labels, pad_to_multiple_of=self.pad_to_multiple_of_labels, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels
        return batch


In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)
data_collator

In [ ]:
from datasets import load_metric

wer_metric = load_metric("wer")

<ipython-input-42-eaec034da022>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  wer_metric = load_metric("wer")


In [ ]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1925: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
import torch
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-arabic-mydata",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  save_steps=10,
  eval_steps=10,
  logging_steps=10,
  learning_rate=3e-4,
  save_total_limit=2,
  logging_dir="./logs",
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)

In [ ]:
# Train the model
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Step,Training Loss,Validation Loss,Wer
10,5.443100,4.938481,1.000000
20,2.248700,3.519940,1.000000
30,1.409300,3.408103,1.000000


/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


TrainOutput(global_step=30, training_loss=3.033714230855306, metrics={'train_runtime': 41.756, 'train_samples_per_second': 2.874, 'train_steps_per_second': 0.718, 'total_flos': 1.32772982837616e+16, 'train_loss': 3.033714230855306, 'epoch': 30.0})

In [ ]:
results = trainer.evaluate()